# Data API for Statistics Denmark (DST)

This notebook provides a short demo on how to call the DST api. 

Another approach for Python is found in Kristian Urup Larsen's repository [here](https://github.com/Kristianuruplarsen/PyDST) that implements functions for pulling data. 

The notebook contains a function that takes as input a table code and certain input parameters. The output is a Pandas DataFrame. The notebook provides an example of how to apply the function and structure the results.

In [8]:
import requests
from io import StringIO

import pandas as pd

dst_api_base = 'https://api.statbank.dk/v1/data/%s/csv?'
options_base =  {'valuePresentation':'Default',
                'timeOrder':'Ascending',
                'allowVariablesInHead':'true'}

def dst_api_dataframe(dataset, options_input={}):    
    '''
    This function produces a dataframe for the desired 
    Statistics Denmark (DST) table. It can be specified 
    to pull particular columns.
    
    Parameters
    ----------
    dataset : str
        DST name/code table to fetch
    options_input : dict, optional
        The extra options can be speficied to pull particular
        column values. E.g. {'ABC':'*'} will return all values 
        of the column 'ABC'. Note specific column values 
        requires separation with comma_char, see example below.
        
    Returns    
    -------
    df_out : pandas.DataFrame
        Table with desired formatted as a pandas DataFrame.
    '''
    
    url = dst_api_base % dataset
    
    options = options_base
    
    for k, v in options_input.items():
        options[k] = v
    
    for k, v in options.items():
        url += '%s=%s&' % (k,v) 
    
    response = requests.get(url[:-1])
    
    df_out = pd.read_csv(StringIO(response.text), sep=';') 
    
    return df_out 
    


# Example of calling API

In the following example we call the API using to fetch the table `KM5` which contain parish level demographic data. We want to pull all parishes for the years 2008-2010 and age 18-65.

In [76]:
parishes = '*' # all parish
ages = '*'
years = range(2008, 2011)

comma_char = '%2C' # separation character for making url

request_options =  {'SOGN': parishes, 
                    'TID': comma_char.join(map(str, years)),
                    'ALDER': comma_char.join(map(str, ages))}
    
data = dst_api_dataframe('KM5', options_input=request_options)

Basic structuring of the dataset which includes renaming to english names. It also preserves the codes for municipality and parish for easy joining with other datasets. Note `SOGNEKODE` means parish code.

In [77]:
data['age'] = data.ALDER.str[:-3].astype(int)

data.drop('ALDER', axis=1, inplace=True)

col_map = {'SOGN':'parish_info', 
           'TID':'year', 
           'INDHOLD':'obs_value'}

data.rename(columns=col_map, inplace=True)
data['KOMKODE'] = data.parish_info.str[:3].astype(int)
data['SOGNEKODE'] = data.parish_info.str[4:8].astype(int)

has_parish = ~ data.SOGNEKODE.isin([0, 9999])

dataset = data[has_parish].drop('parish_info', 1)

dataset.to_csv('danish_parish_demograhic_data.csv', index=False)